In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
#ЗАПУСКАТЬ ТОЛЬКО В СЛУЧАЕ ПОЛНОГО ПЕРЕЗАПУСКА ПАРСЕРА
array = []

In [ ]:
for x in range(0, 4613):
    x_check = x
    offset = x * 25
    url_list = 'https://www.mobygames.com/browse/games/offset,' + str(offset) +'/so,0a/list-games/'
    response_list = requests.get(url_list)
    page_list = BeautifulSoup(response_list.text).find(id = 'mof_object_list').tbody.find_all('tr')
    arr = []

    for j in range(len(page_list)):
        url = page_list[j].a['href']
        response = requests.get(url)
        page_soup = BeautifulSoup(response.text)

        staff = page_soup.find(class_ = 'sideBarTitle')

        if staff is None:
            continue  
        elif staff.h2.text.strip() == 'Credits':
            continue
        elif staff.h2.span is None:
            devs = staff.h2.text.strip()
            staff = 'NA'
            thanks = 'NA'
        else:
            devs = staff.h2.contents[0].strip()
            thanks = staff.h2.span.string.strip().split(', ')[1]
            staff = staff.h2.span.string.strip().split(', ')[0]
               
        title = re.search('[\W\w]*(?= [(]\d{4}[)])', page_soup.title.string).group(0).split(' for ')
        if len(title) < 3:
            title = title[0]
        else:
            title = ' for '.join(title[:-1])

        coreGameRelease = page_soup.find(id = 'coreGameRelease').find_all('div')
        for i in range(len(coreGameRelease)):
            if coreGameRelease[i].text.strip() == 'Released':
                release = coreGameRelease[i + 1].a.string.strip()
            if coreGameRelease[i].text.strip() == 'Platforms' or coreGameRelease[i].text.strip() == 'Platform':
                platforms =  coreGameRelease[i + 1].text.strip()
        
        arr.append([title, release, platforms, devs, staff, thanks])

    array.extend(arr)

In [ ]:
#база АКТИВНО пополняется, поэтому по ходу парсинга могут возникнуть дубликаты
unique = []
for x in array:
    if x not in unique:
        unique.append(x)

In [ ]:
data = pd.DataFrame(unique, columns = ['title', 'release', 'platform', 'n_devs', 'n_staff', 'n_thanks'])
data
# data.to_csv('mg.csv', index = False)

In [ ]:
x_check
# arr